# GoogLeNet 模型（原始版）
在2014年的ImageNet图像识别挑战赛中，一个名叫GoogLeNet的网络结构大放异彩 [1]。它虽然在名字上向LeNet致敬，但在网络结构上已经很难看到LeNet的影子。GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。在随后的几年里，研究人员对GoogLeNet进行了数次改进，这里将介绍这个模型系列的第一个版本。
## Inception块
GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上一节介绍的NiN块相比，这个基础块在结构上更加复杂。

Inception块里有4条并行的线路。前3条线路使用窗口大小分别是1×11×1、3×33×3和5×55×5的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做1×11×1卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用3×33×3最大池化层，后接1×11×1卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。
## GoogLeNet模型
GoogLeNet跟VGG一样，在主体卷积部分中使用5个模块（block），每个模块之间使用步幅为2的3×33×3最大池化层来减小输出高宽。第一模块使用一个64通道的7×77×7卷积层。

第二模块使用2个卷积层：首先是64通道的1×11×1卷积层，然后是将通道增大3倍的3×33×3卷积层。它对应Inception块中的第二条线路。

第三模块串联2个完整的Inception块。第一个Inception块的输出通道数为64+128+32+32=25664+128+32+32=256，其中4条线路的输出通道数比例为64:128:32:32=2:4:1:112832:32=241:164:128:32:32=2:4:1:1。其中第二、第三条线路先分别将输入通道数减小至96/192=1/296/192=1/2和16/192=1/1216/192=1/12后，再接上第二层卷积层。第二个Inception块输出通道数增至128+192+96+64=480128+192+96+64=480，每条线路的输出通道数之比为128:192:96:64=4:6:3:219296:64 = 463:2128:192:96:64=4:6:3:2。其中第二、第三条线路先分别将输入通道数减小至128/256=1/2128/256=1/2和32/256=1/832/256=1/8。

第四模块更加复杂。它串联了5个Inception块，其输出通道数分别是192+208+48+64=512192+208+48+64=512、160+224+64+64=512160+224+64+64=512、128+256+64+64=512128+256+64+64=512、112+288+64+64=528112+288+64+64=528和256+320+128+128=832256+320+128+128=832。这些线路的通道数分配和第三模块中的类似，首先含3×33×3卷积层的第二条线路输出最多通道，其次是仅含1×11×1卷积层的第一条线路，之后是含5×55×5卷积层的第三条线路和含3×33×3最大池化层的第四条线路。其中第二、第三条线路都会先按比例减小通道数。这些比例在各个Inception块中都略有不同。

第五模块有输出通道数为256+320+128+128=832256+320+128+128=832和384+384+128+128=1024384+384+128+128=1024的两个Inception块。其中每条线路的通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。需要注意的是，第五模块的后面紧跟输出层，该模块同NiN一样使用全局平均池化层来将每个通道的高和宽变成1。最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。

GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。本节里我们将输入的高和宽从224降到96来简化计算。下面演示各个模块之间的输出的形状变化。

In [4]:
import tensorflow as tf
import numpy as np
print('Tensorflow version',tf.__version__)
print('Numpy version',np.__version__)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

Tensorflow version 2.2.0
Numpy version 1.18.1


In [9]:
class Inception(tf.keras.layers.Layer):
    def __init__(self,c1, c2, c3, c4):
        super().__init__()
        # 线路1，单1 x 1卷积层
        self.p1_1 = tf.keras.layers.Conv2D(c1, kernel_size=1, activation='relu', padding='same')
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = tf.keras.layers.Conv2D(c2[0], kernel_size=1, padding='same', activation='relu')
        self.p2_2 = tf.keras.layers.Conv2D(c2[1], kernel_size=3, padding='same',
                              activation='relu')
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = tf.keras.layers.Conv2D(c3[0], kernel_size=1, padding='same', activation='relu')
        self.p3_2 = tf.keras.layers.Conv2D(c3[1], kernel_size=5, padding='same',
                              activation='relu')
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = tf.keras.layers.MaxPool2D(pool_size=3, padding='same', strides=1)
        self.p4_2 = tf.keras.layers.Conv2D(c4, kernel_size=1, padding='same', activation='relu')

    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return tf.concat([p1, p2, p3, p4], axis=-1)  # 在通道维上连结输出


class GoogLeNet(Inception):
    
    def __init__(self,c1, c2, c3, c4):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0,axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0,axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]
    
    def see_output_shape(self):
        X = tf.random.uniform(shape=(1, 96, 96, 1))
        for layer in net.layers:
            X = layer(X)
            print(layer.name, 'output shape:\t', X.shape)
            
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.train_images[index],224,224,)
        return resized_images.numpy(), self.train_labels[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.test_images[index],224,224,)
        return resized_images.numpy(), self.test_labels[index]

    
    def build_googlenet(self):
        b1 = tf.keras.models.Sequential()
        b1.add(tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', activation='relu'))
        b1.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))
        
        b2 = tf.keras.models.Sequential()
        b2.add(tf.keras.layers.Conv2D(64, kernel_size=1, padding='same', activation='relu'))
        b2.add(tf.keras.layers.Conv2D(192, kernel_size=3, padding='same', activation='relu'))
        b2.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))
        
        b3 = tf.keras.models.Sequential()
        b3.add(Inception(64, (96, 128), (16, 32), 32))
        b3.add(Inception(128, (128, 192), (32, 96), 64))
        b3.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))
        
        b4 = tf.keras.models.Sequential()
        b4.add(Inception(192, (96, 208), (16, 48), 64))
        b4.add(Inception(160, (112, 224), (24, 64), 64))
        b4.add(Inception(128, (128, 256), (24, 64), 64))
        b4.add(Inception(112, (144, 288), (32, 64), 64))
        b4.add(Inception(256, (160, 320), (32, 128), 128))
        b4.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))
        
        b5 = tf.keras.models.Sequential()
        b5.add(Inception(256, (160, 320), (32, 128), 128))
        b5.add(Inception(384, (192, 384), (48, 128), 128))
        b5.add(tf.keras.layers.GlobalAvgPool2D())

        net = tf.keras.models.Sequential([b1, b2, b3, b4, b5, tf.keras.layers.Dense(10)])
        net.compile(optimizer= tf.keras.optimizers.Adam(lr=1e-7),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
        
        return net
    
    def train_googlenet(self, net, epoch):
        #net.load_weights("./ModelTrain/googlenet_weights.h5")
        
        num_iter = self.num_train//batch_size
        for e in range(epoch):
            for n in range(num_iter):
                x_batch, y_batch = self.get_batch_train(batch_size)
                net.fit(x_batch, y_batch)
                if n%20 == 0:
                    net.save_weights("./ModelTrained/googlenet_weights.h5")


In [10]:
if __name__ == '__main__':
    batch_size = 128
    googlenet = GoogLeNet(64, (96, 128), (16, 32), 32)
    net = googlenet.build_googlenet()
    x_batch, y_batch = googlenet.get_batch_train(batch_size)
    print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)
    #net = googlenet.train_googlenet(net, 5)
    
    #net.load_weights("5.9_googlenet_weights.h5")
    x_test, y_test = googlenet.get_batch_test(2000)
    net.evaluate(x_test, y_test, verbose=2)
## 由于显卡温度过高，人为强制中断， 考虑更强GPU算力

x_batch shape: (128, 224, 224, 1) y_batch shape: (128,)
63/63 - 4s - loss: 6.0425 - accuracy: 0.1075
